## Using the EWXPWSDB Collector Class

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

this walks through process of

- creating a temporary DB
- using the collector class for existing station records to
    - get past data 
    - pull data for short period, e.g. from a scheduler
    - get all recent data, aka catch up data from last record


In [ ]:
%load_ext autoreload
%autoreload 2

## Constants/Config

In [ ]:

station_file = '../data/test_stations.tsv'
station_type = 'DAVIS'

In [ ]:

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from ewxpwsdb.db.models import WeatherStation, Reading, StationType, APIResponse
from ewxpwsdb.db.importdata import import_station_file
from ewxpwsdb.collector import Collector
from ewxpwsdb.time_intervals import UTCInterval
from sqlmodel import select

## create engine temp sqlite database

In [ ]:
# this notebook does not use the .env file or the get_db_url() function for the app's db, 
# instead it creates a new file to be used for a temporary, empty sqlite database, deleted in the end. 
from tempfile import NamedTemporaryFile

temp_sqlitefile = NamedTemporaryFile(suffix = '.db')
temp_db_url = f"sqlite:///{temp_sqlitefile.name}"
engine = get_engine(db_url=temp_db_url)
print(engine)
init_db(engine)
import_station_file(station_file, engine = engine)


In [ ]:
def get_one_station(station_type, engine=engine):
    """ global engine is default"""
    with Session(engine) as session:
        statement = select(WeatherStation).where(WeatherStation.station_type == station_type)
        results = session.exec(statement)
        weather_station = results.first()

    return weather_station

In [ ]:
station = get_one_station(station_type, engine)
print(station.station_code)


In [ ]:
with Session(engine) as session:
    all_stations = session.exec(select(WeatherStation)).fetchall()

for station in all_stations:
    print(f"{station.station_code}: {station.station_type}")


In [ ]:
print(engine)

In [ ]:
collector = Collector(station, engine)
collector.weather_api

In [ ]:
collector._session.commit()

In [ ]:
from datetime import timedelta
interval = UTCInterval.previous_fifteen_minutes()

interval.start = interval.start - timedelta(hours = 2)
interval.end = interval.end - timedelta(hours = 1)
interval

In [ ]:
# call the method to see if it's working
somerex = collector.request_and_store_weather_data_utc(interval)
somerex

In [ ]:
collector.current_responses

In [ ]:
collector.current_readings

### Clean up

remove test database

In [ ]:
if collector:
    collector._session.close()
    collector._engine.dispose()

session.close
engine.dispose()

temp_sqlitefile.close()
collector = None
